In [30]:
%run ./modules.ipynb
frames_relax_tot=ase.io.read('./example_data_MP.xyz', index=':')

In [31]:
n_FPS=2000
n_frames=len(frames_relax_tot)
n_PC=5

In [33]:
magic=np.zeros((len(frames_relax_tot)))
for frame, i in zip(frames_relax_tot, range(len(frames_relax_tot))):
    frame.wrap(eps=1e-12)
    if len(frame)%4==0:
        magic[i]=1

In [34]:
species = list(set([int(n) for frame in frames_relax_tot for n in frame.numbers]))

hypers = dict(
    soap_type="PowerSpectrum",
    interaction_cutoff=3.5,
    max_radial=4,
    max_angular=4,
    gaussian_sigma_type="Constant",
    gaussian_sigma_constant=0.5,
    cutoff_smooth_width=0.5,
    global_species=species,
    expansion_by_species_method="user defined",
    normalize=False,
)
fps_soap = SOAP(
    **hypers,
)
idx_for_fps = np.random.randint(0, len(frames_relax_tot), n_FPS) 
frames_for_fps = [frames_relax_tot[i] for i in idx_for_fps]
X_temp = np.array(
    [
        np.mean(fps_soap.transform([frame]).get_features(fps_soap), axis=0)
        for frame in frames_for_fps
    ]
)

THRESH = 1e-12
high_var_features = np.where(np.var(X_temp - X_temp.mean(axis=0), axis=0) > THRESH)[0]
X_temp = X_temp[:, high_var_features]

X_raw = StandardFlexibleScaler(column_wise=False).fit_transform(X_temp)
del X_temp # save on memory
fps = FPS(n_FPS).fit(X_raw)
del X_raw 
u_species = np.unique(species)
sp_pairs = fps_soap.get_keys(u_species)

coefficient_subselection = np.zeros((n_FPS, 5))
index_mapping = get_power_spectrum_index_mapping(
    sp_pairs, n_max=hypers["max_radial"], l_max=hypers["max_angular"] + 1
)
for fi, i in enumerate(high_var_features[fps.selected_idx_]):
    coefficient_subselection[fi] = [
        index_mapping[i][k] for k in ["a", "b", "n1", "n2", "l"]
    ]
coefficient_subselection_dict = {
    "a": coefficient_subselection[:, 0].tolist(),
    "b": coefficient_subselection[:, 1].tolist(),
    "n1": coefficient_subselection[:, 2].tolist(),
    "n2": coefficient_subselection[:, 3].tolist(),
    "l": coefficient_subselection[:, 4].tolist(),
}

soap = SOAP(**hypers, coefficient_subselection=coefficient_subselection_dict)
idx = np.random.randint(0, len(frames_relax_tot), n_frames) 
my_frames= [frames_relax_tot[i] for i in idx]

magic=magic.reshape(-1, 1)[idx]
               
X_raw = np.zeros((len(my_frames), n_FPS))
for fi, frame in enumerate(my_frames):
    X_raw[fi] = np.mean(soap.transform([frame]).get_features(soap), axis=0)

In [35]:
file=np.savez('./example_SOAP_MP.npz', idx=idx, my_frames=my_frames,
            magic=magic ,X_raw=X_raw, coeff=coefficient_subselection)

/home/egazzarrini/miniconda3/envs/aiida/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
